Model to determine chance of youth getting in trouble at school (p6b39) given...
- whether or not they feel safe at school (k6b1d)
- youth's living arrangements (cp6yloth)
- headaches (p6b16)
- primary caregiver's education (cp6edu)
- days per week they eat with the family (k6c3)

also consider predicting closeness to bio parents
- father(k6c28)
- mother (k6c17)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def corr (x, y):
    print(x+' and '+y+' are corr by '+str(df[x+'_int'].corr(df[y+'_int'])))

df_1=pd.merge(pd.read_csv("FF_Processed_Data_a.csv"),(pd.read_csv("FF_Processed_Data_b.csv")),on='idnum')
print('pair 1...')
df_2=pd.merge((pd.read_csv("FF_Processed_Data_c.csv")),(pd.read_csv("FF_Processed_Data_d.csv")),on='idnum')
print('pair 2...')
df_3=pd.merge(df_1,df_2,on='idnum')
print('pair 3...')
df_4=pd.merge(df_3,(pd.read_csv("FF_Processed_Data_e.csv")),on='idnum')
print('almost...')

df=df_4=pd.merge(df_4,(pd.read_csv("FF_Processed_Data_addi.csv")),on='idnum')
print('Done!')

pair 1...
pair 2...
pair 3...
almost...
Done!


In [24]:
#Determining basic math model
df_safe_best=df[df.k6b1d_int==1]
df_safe_good=df[df.k6b1d_int==2]
df_safe_not=df[df.k6b1d_int==3]
df_safe_worst=df[df.k6b1d_int==4]

for x, y in [('strongly agree', df_safe_best), ('agree', df_safe_good), ('disagree', df_safe_not), ('strongly disagree', df_safe_worst)]:
    chance=50*(y.p6b39_int.mean()-1)
    chanceb=(4-y.k6c28_int.mean())*(100/3)
    print('If they '+x+' to feeling safe at school, {0}% chance of misbehaving there and {1}% closeness to father'.format(chance, chanceb))
    
print('')
    
df_PCG=df[df.cp6yloth_int==1]
df_dad=df[df.cp6yloth_int==2]
df_mgp=df[df.cp6yloth_int==3]
df_pgp=df[df.cp6yloth_int==4]
df_rels=df[df.cp6yloth_int==5]
df_fost=df[df.cp6yloth_int==7]

for x, y in [('with primary care-giver (bio mother)', df_PCG), ('with bio father', df_dad), ('with maternal grandparents', df_mgp), ('with paternal grandparents', df_pgp),
             ('with other relatives', df_rels), ('in foster care', df_fost)]:
    chance=50*(y.p6b39_int.mean()-1)
    chanceb=(4-y.k6c28_int.mean())*(100/3)
    print('If they live '+x+', {0}% chance of misbehaving at school and {1}% closeness to father'.format(chance, chanceb))
    
print('')

df_headache=df[df.p6b16_int==1]
df_no_headache=df[df.p6b16_int==2]

for x, y in [('have frequent headaches', df_headache), ('do not have headaches frequently', df_no_headache)]:
    chance=50*(y.p6b39_int.mean()-1)
    chanceb=(4-y.k6c28_int.mean())*(100/3)
    print('If they '+x+', {0}% chance of misbehaving in school and {1}% closeness to father'.format(chance, chanceb))
    
print('')

for x, y in [('less than HS', df[df.cp6edu_int==1]), ('a HS', df[df.cp6edu_int==2]), ('some college', df[df.cp6edu_int==3]), ('college and/or graduate school', df[df.cp6edu_int==4])]:
    chance=50*(y.p6b39_int.mean()-1)
    chanceb=(4-y.k6c28_int.mean())*(100/3)
    print('If their PCG has had '+x+' education, {0}% chance of misbehaving at school and {1}% closeness to father'.format(chance, chanceb))
    
print('')

list=[(str(x+1), df[df.k6c3_int==x+1]) for x in range(7)]
for x, y in list:
    chance=50*(y.p6b39_int.mean()-1)
    chanceb=(4-y.k6c28_int.mean())*(100/3)
    print('If they have dinner together '+x+' nights per week, {0}% chance of misbehaving at school and {1}% closeness to father'.format(chance, chanceb))

If they strongly agree to feeling safe at school, 11.652542372881358% chance of misbehaving there and 56.28140703517589% closeness to father
If they agree to feeling safe at school, 13.571428571428568% chance of misbehaving there and 53.88888888888889% closeness to father
If they disagree to feeling safe at school, 16.666666666666664% chance of misbehaving there and 54.16666666666667% closeness to father
If they strongly disagree to feeling safe at school, 9.999999999999998% chance of misbehaving there and 33.333333333333336% closeness to father

If they live with primary care-giver (bio mother), 12.30529595015576% chance of misbehaving at school and 54.98154981549816% closeness to father
If they live with bio father, 0.0% chance of misbehaving at school and 33.333333333333336% closeness to father
If they live with maternal grandparents, nan% chance of misbehaving at school and nan% closeness to father
If they live with paternal grandparents, nan% chance of misbehaving at school and na

In [32]:
#above gives summary of chances which are averaged by model
#input is set into model as follows:
    #(a,b,c,d) where a=how safe they feel at school, with 1 being most and 4 being least. Must be integer.
    # b=frequent headaches, where 1=yes and 0=no
    # c=education of primary caregiver, either "less", "HS", "coll", or "grad" (with explanations being shown in summary above)
    # d=number of nights per week when they eat dinner together
    
def predict(a,b,c,d):
    school_a=50*(df[df.k6b1d_int==a].p6b39_int.mean()-1)
    dad_a=(4-df[df.k6b1d_int==a].k6c28_int.mean())*(100/3)
    if b==0:
        b=2
    school_b=50*(df[df.p6b16_int==b].p6b39_int.mean()-1)
    dad_b=(4-df[df.p6b16_int==b].k6c28_int.mean())*(100/3)
    if c=='less':
        c=1
    elif c=='HS':
        c=2
    elif c=='coll':
        c=3
    elif c=='grad':
        c=4
    school_c=50*(df[df.cp6edu_int==c].p6b39_int.mean()-1)
    dad_c=(4-df[df.cp6edu_int==c].k6c28_int.mean())*(100/3)
    school_d=50*(df[df.k6c3_int==d].p6b39_int.mean()-1)
    dad_d=(4-df[df.k6c3_int==d].k6c28_int.mean())*(100/3)
    school=(school_a+school_b+school_c+school_d)/4
    dad=(dad_a+dad_b+dad_c+dad_d)/4
    print('They have a {0}% chance of misbehaving at school and a {1}% closeness to their father.'.format(round(school,2), round(dad,2)))

In [34]:
predict(1, 0, 'grad', 6)
predict(3, 1, 'grad', 7)
predict(2,0,'HS',3)

They have a 10.71% chance of misbehaving at school and a 58.67% closeness to their father.
They have a 12.01% chance of misbehaving at school and a 55.85% closeness to their father.
They have a 11.43% chance of misbehaving at school and a 55.62% closeness to their father.


In [44]:
predict(1, 0, 'grad', 6)
predict(4,1,'less',0)

They have a 10.71% chance of misbehaving at school and a 58.67% closeness to their father.
They have a 12.93% chance of misbehaving at school and a 49.21% closeness to their father.


On account of the lack of correlations within these factors and the simple method, the results are not particularly exciting. Most predictions are around the same, with a 10-12% chance of misbehaving in school and a little over 50% closeness to the father.